# PART Ⅱ
# 구조적 API : DataFrame, SQL, Dataset

# 4장. 구조적 API 개요
- 구조적 API : 비정형 로그 파일부터 반정형 CSV 파일, 매우 정형적인 파케이 파일까지 다양한 유형의 데이터를 처리할 수 있음

- 구조적 API의 세 가지 분산 컬렉션
  - Dataset
  - DataFrame
  - SQL 테이블과 뷰
- 구조적 API 사용할 수 있는 고 : 배치, 스트리밍
  - 구조적 API 활용 시, 배치 작업을 스트리밍 작업으로 손쉽게 변환(반대의 경우도 마찬가지)
  
- 구조적 API : 데이터 흐름을 정의하는 기본 추상화 개념

- 반드시 이해해야 하는 세 가지 기본 개념 설명
  - 타입형(typed)/비타입형(untyped) API의 개념과 차이점
  - 핵심 용어
  - 스파크가 구조적 API의 데이터 흐름을 해석하고 클러스터에서 실행하는 방식

## 4.1 DataFrame과 Dataset
- 스파크는 DataFrame과 Dataset이라는 두 가지 구조화된 컬렉션 개념을 가지고 있음
- DataFrame과 Dataset이 무엇을 나타내는지 먼저 정의
- DataFrame과 Dataset은 잘 정의된 로우와 컬럼을 가지는 분산 테이블 형태의 컬렉션
  - 각 컬럼은 다른 컬럼과 동일한 수의 로우를 가져야 함('값 없음'은 null로 표시함)
  - 컬렉션의 모든 로우는 같은 데이터 타입 정보를 가지고 있어야 함
  - 결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야 하느지 정의하는 지연 연산의 실행 계획이며 불변성을 가짐
- DataFrame에 액션을 호출하면 스파크는 트랜스포메이션을 실제로 실행하고 결과를 반환함
  - 이 과정은 사용자가 원하는 결과를 얻기 위해 로우와 컬럼을 처리하는 방법에 대한 계획을 나타냄

- **NOTE**
  - 기본적으로 테이블과 뷰는 DataFrame과 같음
  - 대신 테이블은 DataFrame 코드 대신 SQL을 사용함
- DataFrame과 Dataset을 조금 더 구체적으로 정의하려면 스키마를 알아야 함
  - 스키마는 분산 컬렉션에 저장할 데이터 타입을 정의하는 방법


## 4.2 스키마
- 스키마
  - DataFrame의 컬럼명과 데이터 타입을 정의함
  - 데이터 소스에서 얻거나 직접 정의할 수 있음
  - 여러 데이터 타입으로 구성되므로 어떤 데이터 타입이 어느 위치에 있는지 정의하는 방법이 필요함
  
## 4.3 스파크의 구조적 데이터 타입 개요
- 스파크 : 사실상 프로그래밍 언어
  - 카탈리스트 엔진 : 실행 계획 수리보가 처리에 사용하는 자체 데이터 타입 정보를 가지고 있는 엔진을 사용함
    - 카탈리스트 엔진의 특징 : 다양한 실행 최적화 기능을 제공
  - 자체 테이터 타입을 지원하는 여러 언어 API와 직접 매핑되며 각 언어에 대한 매핑 테이블을 가지고 있음
    - 파이썬이나 R을 이용해 스파크의 구조적 API를 사용하더라도 대부분의 연산은 파이썬과 R의 데이터 타입이 아닌 스파크의 데이터 타입을 사용함

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sample").master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)
spark.conf.set("spark.sql.shuffle.partitions","5")

In [2]:
#스칼라나 파이썬이 아닌 스파크의 덧셈 연산을 수행함
df=spark.range(500).toDF("number")
df.select(df["number"]+10)

(number + 10)
10
11
12
13
14
15
16
17
18
19


- 스파크에서 덧셈 연산이 수행되는 이유
  - 스파크가 지원하는 언어를 이용해 작성된 표현식을 카탈리스트 엔진에서 스파크의 데이터 타입으로 변환해 명령을 처리하기 때문

### 4.3.1 DataFrame과 Dataset 비교
- 구조적 API에는 '비타입형'인 DataFrame과 '타입형'인 Dataset이 있음
  - DataFrame을 '비타입형'으로 보는 것이 다솨 부정확할 수도 있음
  - DataFrame에도 데이터 타입이 있음
  - 하지만 스키마에 명시된 데이터 타입의 일치 여부를 런타임이 되어서야 확인함
- Dataset은 스키마에 명시된 데이터 타입의 일치 여부를 컴파일 타임에 확인함
  - Dataset은 JVM 기반의 언어인 스칼라와 자바에서만 지원함
  - Dataset의 데이터 타입을 정의하려면 스칼라의 케이스 클래스나 자바 빈을 사용해야 함

- 이 책의 예제는 대부분 DataFrame을 사용함
- 스파크의 DataFrame은 Row 타입으로 구성된 Dataset임
  - Row 타입은 스파크가 사용하는 '연산에 최적화된 인메모리 포맷'의 내부적인 표현 방식임
  - Row 타입을 사용하면 가비지 컬렉션과 객체 초기화 부하가 있는 JVM 데이터 타입을 사용하는 대신 자체 데이터 포맷을 사용하기 때문에 매우 효율적인 연산이 가능함
  - 파이썬이나 R에서는 스파크의 Dataset을 사용할 수 없음
    - 하지만 최적화된 포맷인 Dataframe으로 처리할 수 있음
    
- DataFrame, 스파크의 데이터 타입 그리고 스키마를 완전히 이해하려면 시간이 필요함
  - 지금 기억해야 할 것은 DataFrame을 사용하면 스파크의 최적화된 내부 포맷을 사용할 수 있음
  - 스파크의 최적화된 내부 포맷을 사용하면 스파크가 지원하는 어떤 언어 API를 사용하더라도 동일한 효과 효율성을 얻을 수 있음
  
  
### 4.3.2 컬럼
- 컬럼
  - 단순 데이터 타입 : 정수형이나 문자열
  - 복합 데이터 타입 : 배열이나 맵
  - null 값 표현
- 스파크는 데이터 타입의 모든 정보를 추적하며 다양한 컬럼 변환 방법을 제공함

### 4.3.3 로우
- 로우 : 데이터 레코드
  - DataFrame의 레코드는 Row 타입으로 구성됨
  - 로우는 SQL, RDD, 데이터소스에서 얻거나 직접 만들 수 있음

In [3]:
#range 메서드를 사용해 DataFrame을 생성하는 예제
spark.range(2).collect()
#Row 객체로 이루어진 배열을 반환함

[Row(id=0), Row(id=1)]

### 4.3.4 스파크 데이터 타입
- 스파크는 여러 가지 내부 데이터 타입을 가지고 있음
  - [표4-1-파이썬],[표4-2-스칼라],[표4-3-자바]은 다양한 프로그래밍 언어의 데이터 타입이 스파크의 어떤 데이터 타입과 매핑되는지를 나타냄
  - 표4-1. 파이썬 데이터 타입 매핑
  
  |스파크 데이터 타입|파이썬 데이터 타입|데이터 타입 생성/접근용 API|
  |------------------|------------------|---------------------------|
  |ByteType|int.long.참고: 숫자는 런타임에 1바이트 크리의 부호형 정수(integer)로 변환됨. -128과 127 사이의 값을 가짐|ByteType()|
  |ShortType|int.long.참고: 숫자는 런타임에 2바이트 크기의 부호형 정수로 변환됨. -32768과 32767 사이의 값을 가짐|ShortType()|
  |IntergerType|int.long.참고 : 파이썬은 '정수형' 데이터 타입의 숫자를 관대하게 정의함. 매우 큰 숫자값을 IntegerType()에서 사용하면 스파크 SQL에서 거부할 수 있음. 숫자값이 너무 크면 LongType을 사용해야 함|IntegerType()|
  |LongType|||
  ||||
  ||||
  ||||
  ||||